In [3]:
import pandas as pd
import joblib
import warnings
from sklearn.metrics import mean_absolute_error
warnings.filterwarnings("ignore")

КС 16
Hours 48h
Pmode оба

In [8]:
pmode = "Pin"
pmode = "Pout"
hours = "48h"
ks = "16"
date = "20.12.2021"

In [35]:
# Load the model
def predict(
    ks: str,
    pmode: str,
    hours: str,
    # date: str,
):
    model_path = f"ModelsNewFix/{pmode}_lag_{hours}_КС-{ks}_LGBM_model.joblib"
    loaded_model = joblib.load(model_path)
    # Load training and test data
    train_path = f"data/КС-{ks}_{pmode}_{hours[:-1]}_h.parquet"
    test_path = f"data/КС-{ks}_{pmode}_{hours[:-1]}_h_new.parquet"
    train = pd.read_parquet(train_path)
    test = pd.read_parquet(test_path)
    # Filter test data by date
    # test = test[test["DateTime"].dt.strftime("%d.%m.%Y") == date]
    # if len(test) == 0:
    #     test = train[train["DateTime"].dt.strftime("%d.%m.%Y") == date]
    #     if len(test) == 0:
    #         raise Exception("Error")
    # Prepare training data
    X_train = train.drop(columns=["DateTime", f"{pmode}_target_shift_{hours[:-1]}h_{ks}"])
    y_train = train[f"{pmode}_target_shift_{hours[:-1]}h_{ks}"]
    # Fit the model
    loaded_model.fit(X_train, y_train)
    # Prepare test data
    X_test = test.drop(columns=["DateTime", f"{pmode}_target_shift_{hours[:-1]}h_КС{ks}"])
    # Create date index for the predictions
    date_index = test["DateTime"].apply(lambda x: x + pd.Timedelta(hours=int(hours[:-1])))
    date_index = pd.to_datetime(date_index, errors="coerce").dt.strftime("%Y-%m-%d %H:%M")
    # Make predictions
    result = loaded_model.predict(X_test)
    prediction = pd.concat(
        [
            date_index.reset_index(drop=True),
            pd.DataFrame(result).apply(lambda x: round(x, 3)),
        ],
        axis=1,
    ).rename(columns={0: "Result"})
# Add additional fields
# prediction["mode"] = pmode
# prediction["station"] = ks
# Save prediction to Excel
    return prediction

In [20]:
df = pd.read_csv("inspect/P_in_Plan_Fact.csv", delimiter=";")
df["Data"] = pd.to_datetime(df["Data"])

In [40]:
big_dick = predict("16","Pin","48h")
big_dick["DateTime"] = pd.to_datetime(big_dick["DateTime"])

In [46]:
from datetime import timedelta, datetime

start_date = datetime.strptime('03.01.2021', '%d.%m.%Y')
end_date = datetime.strptime('31.12.2023', '%d.%m.%Y')
best_day = start_date
best_score = 10000
# Iterate over the range of dates
current_date = start_date
while current_date <= end_date:
    try:
        y_pred = big_dick[big_dick["DateTime"].dt.strftime("%d.%m.%Y") == current_date.strftime("%d.%m.%Y")]["Result"]
        # result["DateTime"] = pd.to_datetime(result["DateTime"])
        # y_pred = result[result["DateTime"].dt.strftime("%d.%m.%Y") == current_date.strftime("%d.%m.%Y")]["Result"]
        # df[df["Data"].dt.strftime("%d.%m.%Y %H:%M") == (current_date + timedelta(hours=1)).strftime("%d.%m.%Y %H:%M")][df["Object"] == 1]["P_Plan"]
        y_true = df[df["Data"].dt.strftime("%d.%m.%Y") == current_date.strftime("%d.%m.%Y")][df["Object"] == 2]["P_Fact"]
        # y_true = df[df["Data"].dt.strftime("%d.%m.%Y") == current_date.strftime("%d.%m.%Y")][df["Object"] == 2]["P_Plan"]
        mae_result = mean_absolute_error(y_true, y_pred)
        if mae_result < best_score:
            best_score = mae_result
            best_day = current_date
        current_date += timedelta(days=1)
        # mae_result = mean_absolute_error(y_true, y_pred)
    except:
        continue
print({"mae": best_score, "best_day":best_day, "model": y_pred, "fact": y_true})


In [32]:
y_pred

0     47.587
1     47.587
2     46.316
3     46.316
4     46.291
5     46.291
6     45.761
7     45.761
8     46.187
9     46.187
10    46.337
11    46.337
12    46.742
13    46.742
14    46.882
15    46.882
16    46.825
17    46.825
18    47.340
19    47.340
20    47.338
21    47.338
22    47.338
23    47.338
Name: Result, dtype: float64

In [33]:
y_true

26314   NaN
26315   NaN
26316   NaN
26317   NaN
26318   NaN
26319   NaN
26320   NaN
26321   NaN
26322   NaN
26323   NaN
26324   NaN
26325   NaN
26326   NaN
26327   NaN
26328   NaN
26329   NaN
26330   NaN
26331   NaN
26332   NaN
26333   NaN
26334   NaN
26335   NaN
26336   NaN
26337   NaN
Name: P_Plan, dtype: float64

In [ ]:
best_score, best_day 

In [ ]:
import plotly #for plotting
from plotly import graph_objects as go
from plotly import express as px
from plotly.subplots import make_subplots




In [ ]:
y_pred = big_dick[big_dick["DateTime"].dt.strftime("%d.%m.%Y") == current_date.strftime("%d.%m.%Y")]["Result"]
        # result["DateTime"] = pd.to_datetime(result["DateTime"])
        # y_pred = result[result["DateTime"].dt.strftime("%d.%m.%Y") == current_date.strftime("%d.%m.%Y")]["Result"]
        # df[df["Data"].dt.strftime("%d.%m.%Y %H:%M") == (current_date + timedelta(hours=1)).strftime("%d.%m.%Y %H:%M")][df["Object"] == 1]["P_Plan"]
        y_true = df[df["Data"].dt.strftime("%d.%m.%Y") == current_date.strftime("%d.%m.%Y")][df["Object"] == 2]["P_Fact"]
        # y_true = df[df["Data"].dt.strftime("%d.%m.%Y") == current_date.strftime("%d.%m.%Y")][df["Object"] == 2]["P_Plan"]
        mae_result = mean_absolute_error(y_true, y_pred)
        if mae_result < best_score:
            best_score = mae_result
            best_day = current_date
        current_date += timedelta(days=1)

In [ ]:
print({"mae": best_score, "best_day":best_day, "model": y_pred, "fact": y_true})

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df["Data"], y=y_pred, name="Прогноз"))

fig.add_trace(go.Scatter(x=df["Data"], y=y_plan_true, name="План"))

fig.add_trace(go.Scatter(x=df["Data"], y=y_true, name="Факт"))

fig.update_layout(

    title=dict(text="MAE Pin model= plan=", font=dict(size=18), automargin=True, yref='paper')

)

fig.show()

In [ ]:
def get_series_results_for_date(object_id:str,ks:str, pmode:str, hours:str, date_str: str):
    """
    Params
    ks - номер КС (15, 16, 17, 19 и тд)
    pmode - Pin/Pout
    hours - 48h, 72h, 96h
    date_str - дата
    """
    #* Открываем заготовленный план-факт
    df = pd.read_csv("inspect/P_in_Plan_Fact.csv", delimiter=";")
    df["Data"] = pd.to_datetime(df["Data"])
    date = datetime.strptime(date_str, "%d.%m.%Y")
    result = predict(ks, pmode, hours, (date - timedelta(days=2)).strftime("%d.%m.%Y"))
    result["DateTime"] = pd.to_datetime(result["DateTime"])
    y_pred = result[
        result["DateTime"].dt.strftime("%d.%m.%Y") == date.strftime("%d.%m.%Y")
    ]["Result"]
    # df[df["Data"].dt.strftime("%d.%m.%Y %H:%M") == (current_date + timedelta(hours=1)).strftime("%d.%m.%Y %H:%M")][df["Object"] == 1]["P_Plan"]
    y_true = df[df["Data"].dt.strftime("%d.%m.%Y") == date.strftime("%d.%m.%Y")][
        df["Object"] == 1
    ]["P_Fact"]
    mae_result = mean_absolute_error(y_true, y_pred)
    return {"mae": mae_result, "model": y_pred, "fact": y_true}

In [ ]:
get_series_results_for_date("1","15", "Pin", "48h", "03.01.2021")

In [ ]:
def get_best_result(object_id:str,ks:str, pmode:str, hours:str, start_date: str, end_date: str):
    """
    Params
    ks - номер КС (15, 16, 17, 19 и тд)
    pmode - Pin/Pout
    hours - 48h, 72h, 96h
    start_date - начальная дата 03.01.2021
    end_date - конечная дата 21.01.2021
    """
    # start_date = datetime.strptime('03.01.2021', '%d.%m.%Y')
    # end_date = datetime.strptime('21.01.2021', '%d.%m.%Y')
    best_day = start_date
    best_score = 10000
    # Iterate over the range of dates
    try:
        current_date = start_date
        while current_date <= end_date:
            df = pd.read_csv("inspect/P_in_Plan_Fact.csv", delimiter=";")
            df["Data"] = pd.to_datetime(df["Data"])
            if mae_result < best_score:
                    best_score = mae_result
                    best_day = current_date
            current_date += timedelta(days=1)
    except Exception as e:
        print("Ошибка в значениях: ",e)
        print("Цикл завершен")
    return best_day, best_score
